In [ ]:
# % pip install sklearn

In [ ]:
import torch
torch.manual_seed(42) # Set Seed for reproducibility
import conllu
from io import open
from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator, Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import csv
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

## Plot FFNN models

In [ ]:
# Read the CoNLL-U file
def read_conllu_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.read()
    return data

# Parse the CoNLL-U data
def parse_conllu_data(conllu_data):
    return conllu.parse(conllu_data)

# Extract sentences and POS tags
def extract_sentences_and_postags(parsed_data):
    sentences_and_postags = []
    for tokenList in parsed_data:
        sentence = []
        postagList = []
        for token in tokenList:
            # print(token)
            word = token['form']
            postag = token['upostag']
            # filter out punctuation
            if postag != "PUNCT":
                sentence.append(word)
                postagList.append(postag)
        # if len(sentence) == 0 or len(postagList) == 0:
        #     print("Empty sentence or postagList")
        sentences_and_postags.append((sentence, postagList))
    return sentences_and_postags

def getSentencesAndPosTags(conllu_file_path):
    conllu_data = read_conllu_file(conllu_file_path)
    parsed_data = parse_conllu_data(conllu_data)
    # print(parsed_data)
    return extract_sentences_and_postags(parsed_data)

# sentences_and_postags = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-train.conllu")
# # Print the list of words and POS tags
# for sentence, postagList in sentences_and_postags:
#     print(f"sentence: {sentence}, postagList: {postagList}")
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"
SPECIAL_TOKENS = [START_TOKEN, END_TOKEN, UNKNOWN_TOKEN, PAD_TOKEN]
SPECIAL_POS_TAG = "SPECIAL"

class ffnDataset(Dataset):
    def __init__(self, data : list[tuple[list[str], list[str]]] , pOfNN: int , sOfNN: int, wordVocabulary:Vocab|None=None , tagVocabulary:Vocab|None=None):
        """Initialize the dataset. Setup Code goes here"""
        self.pOfNN = pOfNN
        self.sOfNN = sOfNN
        # self.sentences = [i[0] for i in data] # list of sentences
        self.chunks = []
        for i in range(len(data)):
            sentence = self.insertStartEndTokens(data[i][0])
            chunksList = [sentence[i: i+pOfNN+sOfNN+1] for i in range(len(sentence)-pOfNN-sOfNN)]
            self.chunks.extend(chunksList)
        self.labels = []
        for i in range(len(data)):
            sentenceTags = data[i][1]
            self.labels.extend(sentenceTags)

        if wordVocabulary is None:
            sentences = [i[0] for i in data] # list of sentences
            self.wordVocabulary = build_vocab_from_iterator(sentences, specials= SPECIAL_TOKENS, min_freq= 2) # use min_freq for handling unkown words better
            self.wordVocabulary.set_default_index(self.wordVocabulary[UNKNOWN_TOKEN])
        else:
            # if vocabulary provided use that
            self.wordVocabulary = wordVocabulary
        if tagVocabulary is None:
            self.tagVocabulary = build_vocab_from_iterator([self.labels], specials= [SPECIAL_POS_TAG])
            self.tagVocabulary.set_default_index(self.tagVocabulary[SPECIAL_POS_TAG]) # index for unknown tags
        else:
            # if vocabulary providePosDatasetd use that
            self.tagVocabulary = tagVocabulary


    def __len__(self) -> int:
        """Returns number of datapoints."""
        return len(self.chunks)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """Get the datapoint at `index`."""
        posTagIndex = self.tagVocabulary[self.labels[index]]
        # # create a one hot vector for the pos tag
        # posTagVector = torch.zeros(len(self.tagVocabulary))
        # posTagVector[posTagIndex] = 1
        posTagVector = posTagIndex # not converting to vector
        return torch.tensor(self.wordVocabulary.lookup_indices(self.chunks[index])), torch.tensor(posTagVector)

    def insertStartEndTokens(self, sentence: list[str]) -> list[str]:
        """Add start and end tokens to the sentence"""
        return [START_TOKEN]*self.pOfNN + sentence + [END_TOKEN]*self.sOfNN

    def createCSV(self, filename: str):
        # create a csv file with the chunks and labels to test if the dataset is created correctly
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["chunks", "labels"])
            for i in range(len(self.chunks)):
                writer.writerow([self.chunks[i], self.labels[i]])

    def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
        # """Given a list of datapoints, batch them together"""
        # sentences = [i[0] for i in batch]
        # labels = [i[1] for i in batch]
        # padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.vocabulary[PAD_TOKEN]) # pad sentences with pad token id
        # padded_labels = pad_sequence(sentences, batch_first=True, padding_value=torch.tensor(0)) # pad labels with 0 because pad token cannot be entities

        # return padded_sentences, padded_labels
        pass
class FFN(torch.nn.Module):
    def __init__(self, vocabularySize: int, embeddingSize: int, p: int, s: int, tagVocabSize: int, activationFunction: torch.nn.Module, hiddenLayerSizes: list[int] = []):
        super(FFN, self).__init__()
        self.embeddingModule = torch.nn.Embedding(
            vocabularySize, embeddingSize)
        # self.embeddingModule = torch.nn.Embedding.from_pretrained(pretrainedEmbeddings)
        # self.embeddingModule.weight.requires_grad = False
        self.p = p
        self.s = s
        self.vocabularySize = vocabularySize
        self.embeddingSize = embeddingSize
        self.tagVocabSize = tagVocabSize
        self.activationFunction = activationFunction
        self.numOfInputWords = p + s + 1
        self.inputSize = embeddingSize*self.numOfInputWords
        self.outputSize = tagVocabSize
        self.hiddenLayerSizes = hiddenLayerSizes
        moduleList = []
        for i in range(len(self.hiddenLayerSizes)):
            if i == 0:
                moduleList.append(torch.nn.Linear(
                    self.inputSize, self.hiddenLayerSizes[i]))
            else:
                moduleList.append(torch.nn.Linear(
                    self.hiddenLayerSizes[i-1], self.hiddenLayerSizes[i]))
            moduleList.append(activationFunction)
        moduleList.append(torch.nn.Linear(
            self.hiddenLayerSizes[-1], self.outputSize))
        self.posTagger = torch.nn.Sequential(*moduleList)

    # wordIndices is a tensor of indices of words in the chunk
    def forward(self, wordIndices: torch.tensor):
        wordEmbeddings = self.embeddingModule(wordIndices)
        wordEmbeddings = wordEmbeddings.view(wordEmbeddings.size(0), -1)
        return self.posTagger(wordEmbeddings)

evalRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-dev.conllu")
testRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-test.conllu")
wordVocab = torch.load("ffnTrainWordVocab.pth")
tagVocab = torch.load("ffnTrainTagVocab.pth")
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
def plotConfusionMatrix(matrix,classes,modelPath):
    plt.figure(figsize=(10, 7))
    sns.heatmap(matrix, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix of '+modelPath)
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()


## Val set metrics

In [ ]:
ffnModelList = ["ffnDiffConf1.pt", "ffnDiffConf2.pt","ffnDiffConf3.pt"]
for modelPath in ffnModelList:
        ffnModel = torch.load(modelPath,map_location=torch.device(device)) # if loading from gpu to cpu
        ffnModel = ffnModel.to(device)
        ffnModel.eval()
        evalDataset = ffnDataset(evalRawData ,pOfNN= ffnModel.p, sOfNN= ffnModel.s, wordVocabulary=wordVocab, tagVocabulary=tagVocab)
        evalDataloader = DataLoader(evalDataset, batch_size=64, shuffle=False)
        # lossFn = torch.nn.CrossEntropyLoss()
        # valLoss = 0
        # correct = 0
        # total = 0
        allPredictions = []
        allTrueLabels = []
        # turn off gradients
        with torch.no_grad():
                for batchNum, (inputChunks , tags) in enumerate(evalDataloader):
                        # print(f"Batch: {batchNum}")
                        # Send data to device
                        inputChunks = inputChunks.to(device)
                        tags = tags.to(device)
                        # Forward Pass
                        pred = ffnModel(inputChunks)
                        # loss = lossFn(pred, tags)
                        # valLoss += loss.item()
                        # Get Predictions
                        pred = torch.argmax(pred, dim=1)
                        # tagIndices = torch.argmax(tags, dim=1)
                        tagIndices = tags
                        # total += tags.size(0)
                        # correct += (pred == tagIndices).sum().item()
                # # Print test Loss and Accuracy
                # print(f'val Loss: {valLoss/len(evalDataloader)}, val Accuracy: {(correct/total)*100} %'
                        # move data to cpu
                        pred = pred.cpu().numpy()
                        tagIndices = tagIndices.cpu().numpy()
                        allPredictions.extend(pred)
                        allTrueLabels.extend(tagIndices)

        # get evaluation metrics
        accuracy = accuracy_score(tagIndices, pred)
        recall_micro = recall_score(tagIndices, pred, average='micro')
        recall_macro = recall_score(tagIndices, pred, average='macro')
        f1_micro = f1_score(tagIndices, pred, average='micro')
        f1_macro = f1_score(tagIndices, pred, average='macro')
        conf_matrix = confusion_matrix(tagIndices, pred)

        # print evaluation metrics
        print(f"ffnModel: {modelPath} ,Accuracy: {accuracy}, Recall Micro: {recall_micro}, Recall Macro: {recall_macro}, F1 Micro: {f1_micro}, F1 Macro: {f1_macro}")
        # plot confusion matrix
        classes = [i for i in range(0, len(tagVocab))]
        plotConfusionMatrix(conf_matrix, classes, modelPath)



## test set eval for best Model

In [ ]:
# here best model is ffnDiffConf1.pt
modelPath = "ffnDiffConf1.pt"
ffnModel = torch.load(modelPath,map_location=torch.device(device)) # if loading from gpu to cpu
ffnModel = ffnModel.to(device)
ffnModel.eval()
testDataset = ffnDataset(testRawData ,pOfNN= ffnModel.p, sOfNN= ffnModel.s, wordVocabulary=wordVocab, tagVocabulary=tagVocab)
testDataloader = DataLoader(testDataset, batch_size=64, shuffle=False)
# lossFn = torch.nn.CrossEntropyLoss()
# valLoss = 0
# correct = 0
# total = 0
allPredictions = []
allTrueLabels = []
# turn off gradients
with torch.no_grad():
    for batchNum, (inputChunks , tags) in enumerate(testDataloader):
        # print(f"Batch: {batchNum}")
        # Send data to device
        inputChunks = inputChunks.to(device)
        tags = tags.to(device)
        # Forward Pass
        pred = ffnModel(inputChunks)
        # loss = lossFn(pred, tags)
        # valLoss += loss.item()
        # Get Predictions
        pred = torch.argmax(pred, dim=1)
        # tagIndices = torch.argmax(tags, dim=1)
        tagIndices = tags
        # total += tags.size(0)
        # correct += (pred == tagIndices).sum().item()
# # Print test Loss and Accuracy
# print(f'val Loss: {valLoss/len(evalDataloader)}, val Accuracy: {(correct/total)*100} %'
        # move data to cpu
        pred = pred.cpu().numpy()
        tagIndices = tagIndices.cpu().numpy()
        allPredictions.extend(pred)
        allTrueLabels.extend(tagIndices)

# get evaluation metrics
accuracy = accuracy_score(tagIndices, pred)
recall_micro = recall_score(tagIndices, pred, average='micro')
recall_macro = recall_score(tagIndices, pred, average='macro')
f1_micro = f1_score(tagIndices, pred, average='micro')
f1_macro = f1_score(tagIndices, pred, average='macro')
conf_matrix = confusion_matrix(tagIndices, pred)

# print evaluation metrics
print(f"ffnModel: {modelPath} ,Accuracy: {accuracy}, Recall Micro: {recall_micro}, Recall Macro: {recall_macro}, F1 Micro: {f1_micro}, F1 Macro: {f1_macro}")
# plot confusion matrix
classes = [i for i in range(0, len(tagVocab))]
plotConfusionMatrix(conf_matrix, classes, "ffnBest")

## P,S vs Val set accuracy

In [ ]:
ps = [0,1,2,3,4]
accuracyList = []
ffnModelList = ["ffnPS0.pt", "ffnPS1.pt", "ffnPS2.pt", "ffnPS3.pt", "ffnPS4.pt"]
for modelPath in ffnModelList:
        ffnModel = torch.load(modelPath,map_location=torch.device(device)) # if loading from gpu to cpu
        ffnModel = ffnModel.to(device)
        ffnModel.eval()
        evalDataset = ffnDataset(evalRawData ,pOfNN= ffnModel.p, sOfNN= ffnModel.s, wordVocabulary=wordVocab, tagVocabulary=tagVocab)
        evalDataloader = DataLoader(evalDataset, batch_size=64, shuffle=False)
        # lossFn = torch.nn.CrossEntropyLoss()
        # valLoss = 0
        # correct = 0
        # total = 0
        allPredictions = []
        allTrueLabels = []
        # turn off gradients
        with torch.no_grad():
                for batchNum, (inputChunks , tags) in enumerate(evalDataloader):
                        # print(f"Batch: {batchNum}")
                        # Send data to device
                        inputChunks = inputChunks.to(device)
                        tags = tags.to(device)
                        # Forward Pass
                        pred = ffnModel(inputChunks)
                        # loss = lossFn(pred, tags)
                        # valLoss += loss.item()
                        # Get Predictions
                        pred = torch.argmax(pred, dim=1)
                        # tagIndices = torch.argmax(tags, dim=1)
                        tagIndices = tags
                        # total += tags.size(0)
                        # correct += (pred == tagIndices).sum().item()
                # # Print test Loss and Accuracy
                # print(f'val Loss: {valLoss/len(evalDataloader)}, val Accuracy: {(correct/total)*100} %'
                        # move data to cpu
                        pred = pred.cpu().numpy()
                        tagIndices = tagIndices.cpu().numpy()
                        allPredictions.extend(pred)
                        allTrueLabels.extend(tagIndices)

        # get evaluation metrics
        accuracy = accuracy_score(tagIndices, pred)
        accuracyList.append(accuracy)
# plot accuracy vs ps
plt.plot(ps, accuracyList)
plt.xlabel('p,s')
plt.ylabel('Accuracy')
plt.title('Accuracy vs p,s')
plt.show()

## Analysis (10 marks)

Based on the observed metrics and graphs, draw conclusions from them and
explain your results.

## Report content(pdf) 20 marks

1. pics of graphs and evaluation metrics
2. analysis of the results)

# Plot LSTM models

In [ ]:
import torch.nn.functional as F


In [ ]:
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"
SPECIAL_TOKENS = [START_TOKEN, END_TOKEN, UNKNOWN_TOKEN, PAD_TOKEN]
UNK_POS_TAG = "UNKTAG"
SPECIAL_POS_TAG = "SPECIAL"

class lstmDataset(Dataset):
    def __init__(self, data : list[tuple[list[str], list[str]]] , wordVocabulary:Vocab|None=None , tagVocabulary:Vocab|None=None):
        """Initialize the dataset. Setup Code goes here"""
        self.sentences = [i[0] for i in data] # list of sentences
        self.labels = [i[1] for i in data] # list of labels
        if wordVocabulary is None:
            sentences = [i[0] for i in data] # list of sentences
            self.wordVocabulary = build_vocab_from_iterator(sentences, specials= SPECIAL_TOKENS, min_freq= 2) # use min_freq for handling unkown words better
            self.wordVocabulary.set_default_index(self.wordVocabulary[UNKNOWN_TOKEN])
        else:
            # if vocabulary provided use that
            self.wordVocabulary = wordVocabulary
        if tagVocabulary is None:
            self.tagVocabulary = build_vocab_from_iterator(self.labels, specials= [UNK_POS_TAG,SPECIAL_POS_TAG])
            self.tagVocabulary.set_default_index(self.tagVocabulary[UNK_POS_TAG]) # index for unknown tags
        else:
            # if vocabulary provided use that
            self.tagVocabulary = tagVocabulary
        # # represent tagVocab as one hot vectors
        # self.tagVocabVectors = torch.eye(len(self.tagVocabulary))


    def __len__(self) -> int:
        """Returns number of datapoints."""
        return len(self.sentences)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """Get the datapoint at `index`."""
        return torch.tensor(self.wordVocabulary.lookup_indices(self.sentences[index])), torch.tensor(self.tagVocabulary.lookup_indices(self.labels[index]))

    # def insertStartEndTokens(self, sentence: list[str]) -> list[str]:
    #     """Add start and end tokens to the sentence"""
    #     return [START_TOKEN]*self.pOfNN + sentence + [END_TOKEN]*self.sOfNN

    def createCSV(self, filename: str):
        # create a csv file with the chunks and labels to test if the dataset is created correctly
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["chunks", "labels"])
            for i in range(len(self.sentences)):
                writer.writerow([self.sentences[i], self.labels[i]])

    def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
        """Given a list of datapoints, batch them together"""
        sentences = [i[0] for i in batch]
        labels = [i[1] for i in batch]
        padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.wordVocabulary[PAD_TOKEN]) # pad sentences with pad token id
        padded_labels = pad_sequence(labels, batch_first=True, padding_value= self.tagVocabulary[SPECIAL_POS_TAG]) # pad labels with special_posTag
        # padded_labels = []
        # for i in range(len(padded_sentences)):
        #     sentence = padded_sentences[i]
        #     sentencesLabels = labels[i]
        #     padded_sentence_labels = []
        #     for j in range(len(sentence)):
        #         word = sentence[j]
        #         if word == self.wordVocabulary[PAD_TOKEN]:
        #             padded_sentence_labels.append(self.tagVocabVectors[self.tagVocabulary[SPECIAL_POS_TAG]])
        #         else:
        #             padded_sentence_labels.append(sentencesLabels[j])
        #     padded_labels.append(torch.tensor(padded_sentence_labels))
        return padded_sentences, padded_labels


class LSTM(torch.nn.Module):
    def __init__(self, wordVocabularySize: int, embeddingSize: int, hiddenDim: int, tagVocabSize: int, numLayers: int, classifyHiddenDim:int, activationFunction: torch.nn.Module,isBidirectional: bool) -> None:
        super().__init__()
        self.embeddingSize = embeddingSize
        self.hiddenDim = hiddenDim
        self.numLayers = numLayers
        self.wordVocabularySize = wordVocabularySize
        self.embeddingModule = torch.nn.Embedding(wordVocabularySize, embeddingSize)
        # self.embeddingModule = torch.nn.Embedding.from_pretrained(pretrainedEmbeddings)
        # self.embeddingModule.weight.requires_grad = False
        self.outputSize = tagVocabSize
        self.tagVectors = torch.eye(self.outputSize)
        self.classifyHiddenDim = classifyHiddenDim
        self.activationFunction = activationFunction
        self.isBidirectional = isBidirectional
        if isBidirectional == False:
          self.lstm = torch.nn.LSTM(self.embeddingSize,self.hiddenDim,self.numLayers)
          finalModuleList = []
          finalModuleList.append(torch.nn.Linear(self.hiddenDim,self.classifyHiddenDim))
          finalModuleList.append(self.activationFunction)
          finalModuleList.append(torch.nn.Linear(self.classifyHiddenDim,self.outputSize))
          self.finalModule = torch.nn.Sequential(*finalModuleList)
        else:
          self.lstm = torch.nn.LSTM(self.embeddingSize,self.hiddenDim,self.numLayers,bidirectional=True)
          # self.finalModule = torch.nn.Linear(self.hiddenDim*2,self.outputSize) # *2 because of bidirectional
          finalModuleList = []
          finalModuleList.append(torch.nn.Linear(self.hiddenDim*2,self.classifyHiddenDim))
          finalModuleList.append(self.activationFunction)
          finalModuleList.append(torch.nn.Linear(self.classifyHiddenDim,self.outputSize))
          self.finalModule = torch.nn.Sequential(*finalModuleList)


    def forward(self, sentence: torch.tensor):
        wordEmbeddings = self.embeddingModule(sentence)
        lstm_ouput, hid = self.lstm(wordEmbeddings.view(len(sentence), 1, -1))
        tags = self.finalModule(lstm_ouput.view(len(sentence), -1))
        tag_scores = F.log_softmax(tags,dim=1)
        return tag_scores

    def getTagVectors(self, tags: torch.tensor):
        return self.tagVectors[tags]
wordVocab = torch.load("lstmTrainWordVocab.pth")
tagVocab = torch.load("lstmTrainTagVocab.pth")


## val evaluation

In [ ]:
lstmModelList = ["lstmConfig0.pt", "lstmConfig1.pt","lstmConfig2.pt"]
for modelPath in lstmModelList:
        lstmModel = torch.load(modelPath,map_location=torch.device(device)) # if loading from gpu to cpu
        lstmModel = lstmModel.to(device)
        lstmModel.eval()
        evalDataset = lstmDataset(evalRawData , wordVocabulary=wordVocab, tagVocabulary=tagVocab)
        evalDataloader = DataLoader(evalDataset, batch_size=1, shuffle=False,collate_fn= evalDataset.collate)
        # lossFn = torch.nn.CrossEntropyLoss()
        # valLoss = 0
        # correct = 0
        # total = 0
        allPredictions = []
        allTrueLabels = []
        # turn off gradients
        with torch.no_grad():
                for batchNum, (inputSentences , tags) in enumerate(evalDataloader):
                        # print(f"Batch: {batchNum}")
                        # Send data to device
                        inputSentences = inputSentences[0]
                        tags = tags[0]
                        inputSentences = inputSentences.to(device)
                        tags = tags.to(device)
                        # Forward Pass
                        pred = lstmModel(inputSentences)
                        # loss = lossFn(pred, tags)
                        # valLoss += loss.item()
                        # Get Predictions
                        pred = torch.argmax(pred, dim=1)
                        # tagIndices = torch.argmax(tags, dim=1)
                        tagIndices = tags
                        # total += tags.size(0)
                        # correct += (pred == tagIndices).sum().item()
                # # Print test Loss and Accuracy
                # print(f'val Loss: {valLoss/len(evalDataloader)}, val Accuracy: {(correct/total)*100} %'
                        # move data to cpu
                        pred = pred.cpu().numpy()
                        tagIndices = tagIndices.cpu().numpy()
                        allPredictions.extend(pred)
                        allTrueLabels.extend(tagIndices)

        # get evaluation metrics
        accuracy = accuracy_score(tagIndices, pred)
        recall_micro = recall_score(tagIndices, pred, average='micro')
        recall_macro = recall_score(tagIndices, pred, average='macro')
        f1_micro = f1_score(tagIndices, pred, average='micro')
        f1_macro = f1_score(tagIndices, pred, average='macro')
        conf_matrix = confusion_matrix(tagIndices, pred)

        # print evaluation metrics
        print(f"lstmModel: {modelPath} ,Accuracy: {accuracy}, Recall Micro: {recall_micro}, Recall Macro: {recall_macro}, F1 Micro: {f1_micro}, F1 Macro: {f1_macro}")
        # plot confusion matrix
        classes = [i for i in range(0, len(tagVocab))]
        plotConfusionMatrix(conf_matrix, classes, modelPath)



## test set eval on best model

In [ ]:
modelPath = "lstmConfig1.pt"
lstmModel = torch.load(modelPath,map_location=torch.device(device)) # if loading from gpu to cpu
lstmModel = lstmModel.to(device)
lstmModel.eval()
testDataset = lstmDataset(testRawData , wordVocabulary=wordVocab, tagVocabulary=tagVocab)
testDataloader = DataLoader(testDataset, batch_size=1, shuffle=False,collate_fn= evalDataset.collate)
# lossFn = torch.nn.CrossEntropyLoss()
# valLoss = 0
# correct = 0
# total = 0
allPredictions = []
allTrueLabels = []
# turn off gradients
with torch.no_grad():
        for batchNum, (inputSentences , tags) in enumerate(testDataloader):
                # print(f"Batch: {batchNum}")
                # Send data to device
                inputSentences = inputSentences[0]
                tags = tags[0]
                inputSentences = inputSentences.to(device)
                tags = tags.to(device)
                # Forward Pass
                pred = lstmModel(inputSentences)
                # loss = lossFn(pred, tags)
                # valLoss += loss.item()
                # Get Predictions
                pred = torch.argmax(pred, dim=1)
                # tagIndices = torch.argmax(tags, dim=1)
                tagIndices = tags
                # total += tags.size(0)
                # correct += (pred == tagIndices).sum().item()
        # # Print test Loss and Accuracy
        # print(f'val Loss: {valLoss/len(evalDataloader)}, val Accuracy: {(correct/total)*100} %'
                # move data to cpu
                pred = pred.cpu().numpy()
                tagIndices = tagIndices.cpu().numpy()
                allPredictions.extend(pred)
                allTrueLabels.extend(tagIndices)

# get evaluation metrics
accuracy = accuracy_score(tagIndices, pred)
recall_micro = recall_score(tagIndices, pred, average='micro')
recall_macro = recall_score(tagIndices, pred, average='macro')
f1_micro = f1_score(tagIndices, pred, average='micro')
f1_macro = f1_score(tagIndices, pred, average='macro')
conf_matrix = confusion_matrix(tagIndices, pred)

# print evaluation metrics
print(f"lstmModel: {modelPath} ,Accuracy: {accuracy}, Recall Micro: {recall_micro}, Recall Macro: {recall_macro}, F1 Micro: {f1_micro}, F1 Macro: {f1_macro}")
# plot confusion matrix
classes = [i for i in range(0, len(tagVocab))]
plotConfusionMatrix(conf_matrix, classes, modelPath)



## Report content(pdf) 20 marks

1. pics of graphs and evaluation metrics
2. analysis of the results)